In [9]:
from numpy.core.numeric import tensordot
import pyrealsense2 as rs
import numpy as np
import cv2
import os
import time
import mediapipe as mp
import math
import csv
import plotly.offline as po
import plotly.graph_objects as go
import pandas as pd
import pyautogui
import datetime
import copy
import traceback
import collections

# 画面表示保存用リスト
screenshots = []
color_images = []
depth_colormaps = []

# ランドマーク保存用リスト
landmarks = []

# depth_image保存用リスト
depth_images = []

# XYZ保存用およびCSV一括保存用リスト
X, Y, Z = 0, 0, 0
XYZlist = []
XYZlists = []

# キック軌道表示用キュー（とりあえず過去10フレーム分）
kick_frames = 10
q = collections.deque([], kick_frames)

# Enterキー押下判定用フラグ
flag = False

# 棒人間のライン座標設定用
def F(fig, i, a, b):
    if (marker_x[a] == 0 and marker_y[a] == 0 and marker_z[a] == 0) or \
       (marker_x[b] == 0 and marker_y[b] == 0 and marker_z[b] == 0):
        fig.data[i].x = 0., 0.
        fig.data[i].y = 0., 0.
        fig.data[i].z = 0., 0.
    else:
        fig.data[i].x = marker_z[a], marker_z[b]
        fig.data[i].y = marker_x[a], marker_x[b]
        fig.data[i].z = marker_y[a], marker_y[b]
        
# キック軌道座標設定用
def kick(n, fig):
    x_list = []
    y_list = []
    z_list = []
    
    for i in q:
        x_list.append(i["z"])
        y_list.append(i["x"])
        z_list.append(i["y"])
            
    fig.data[n].x = x_list
    fig.data[n].y = y_list
    fig.data[n].z = z_list
    
def calcXYZ(XYZlist, depth_image, landmark):    
    for i in range(33):
        try:
            # ランドマークのxy座標を取得する
            #（ここはランドマークが取得されていれば失敗しない）
            lmk = landmark[i]
            dist = {"x": lmk.x, "y": lmk.y}                        

            # xy座標からZ座標を取得する
            #（ここはget_distance関数の性質上失敗することがある）
            # →fps向上のためにdepth_imageから取得する方法に変更
            try:
                dist["Z"] = depth_image[int(dist["y"] * h), int(dist["x"] * w)] / 1000

            except:
                # とりあえず0を入れておく（要検討）
                dist["Z"] = 0

            # XYZ座標に変換する
            X = dist["Z"] / fx * (dist["x"] * w - w / 2)
            Y = dist["Z"] / fy * (dist["y"] * h - h / 2)
            Z = dist["Z"]
        
        except:
            # ランドマーク取得不可のときは仮に原点座標を入れておく
            X = 0
            Y = 0
            Z = 0
                                    
        #X->Z Y->X Z->Y
        XYZlist.extend([Z, X, Y])
        
        # マーカーリストに追加する
        marker_x.append(X)
        marker_y.append(Y)
        marker_z.append(Z)
        
    # キューに右足のXYZを追加する（右足のつま先なら32番のランドマーク）    
    q.append({"x": marker_x[32], "y": marker_y[32], "z": marker_z[32]})

def figBatchUpdate(fig):
    with fig.batch_update():
        # 頭部
        F(fig, 1, 0, 1)
        F(fig, 2, 1, 2)
        F(fig, 3, 2, 3)
        F(fig, 4, 0, 4)
        F(fig, 5, 4, 5)
        F(fig, 6, 5, 6)
        F(fig, 7, 3, 7)
        F(fig, 8, 6, 8)
        F(fig, 10, 9, 10)
    
        # 上半身
        F(fig, 12, 11, 12)
        F(fig, 13, 11, 13)
        F(fig, 14, 12, 14)
        F(fig, 15, 13, 15)
        F(fig, 16, 14, 16)
        F(fig, 17, 15, 17)
        F(fig, 18, 16, 18)
        F(fig, 19, 15, 19)
        F(fig, 20, 16, 20)
        F(fig, 21, 15, 21)
        F(fig, 22, 16, 22)
        F(fig, 33, 18, 20)
        F(fig, 34, 17, 19)
    
        # 下半身
        F(fig, 23, 11, 23)
        F(fig, 24, 12, 24)
        F(fig, 35, 23, 24)
        F(fig, 25, 23, 25)
        F(fig, 26, 24, 26)
        F(fig, 27, 25, 27)
        F(fig, 28, 26, 28)
        F(fig, 29, 27, 29)
        F(fig, 30, 28, 30)
        F(fig, 31, 27, 31)
        F(fig, 32, 28, 32)
        
        # キック軌道
        kick(36, fig)

# ここからメイン処理
po.init_notebook_mode(connected = True)
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# make dir
timestr = time.strftime("%y%m%d-%H%M%S")
dirname = "recordimages/%s" % timestr
dirname2 = "plotlyimages/%s" % timestr
os.makedirs(dirname)
os.makedirs(dirname2)

# ストリーム(IR/Color/Depth)の設定
w = 1280
h = 720
config = rs.config()
config.enable_stream(rs.stream.color, w, h, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, w, h, rs.format.z16, 30)

# ストリーミング開始
pipeline = rs.pipeline()
profile = pipeline.start(config)

# Alignオブジェクト生成
align_to = rs.stream.color
align = rs.align(align_to)

try:
    count = -1

    with open('%s.csv' % timestr, 'w', newline="") as f:
        writer = csv.writer(f)

        # CSVファイルの1行目を書き込み
        csv_header = ["FrameNo.", "Time",
                      "0_x", "0_y", "0_z", "1_x", "1_y", "1_z", "2_x", "2_y", "2_z",
                      "3_x", "3_y", "3_z", "4_x", "4_y", "4_z", "5_x", "5_y", "5_z",
                      "6_x", "6_y", "6_z", "7_x", "7_y", "7_z", "8_x", "8_y", "8_z",
                      "9_x", "9_y", "9_z", "10_x", "10_y", "10_z", "11_x", "11_y", "11_z",
                      "12_x", "12_y", "12_z", "13_x", "13_y", "13_z", "14_x", "14_y", "14_z",
                      "15_x", "15_y", "15_z", "16_x", "16_y", "16_z", "17_x", "17_y", "17_z",
                      "18_x", "18_y", "18_z", "19_x", "19_y", "19_z", "20_x", "20_y", "20_z",
                      "21_x", "21_y", "21_z", "22_x", "22_y", "22_z", "23_x", "23_y", "23_z",
                      "24_x", "24_y", "24_z", "25_x", "25_y", "25_z", "26_x", "26_y", "26_z",
                      "27_x", "27_y", "27_z", "28_x", "28_y", "28_z", "29_x", "29_y", "29_z",
                      "30_x", "30_y", "30_z", "31_x", "31_y", "31_z", "32_x", "32_y", "32_z"
                      ]
        writer.writerow(csv_header)
        
        # ライン
        data = []        
        for i in range(36):
            data.append(go.Scatter3d(
                x =  [0, 0],
                y =  [0, 0],
                z =  [0, 0],
                mode = 'lines+markers',
                marker = dict(
                    color = 'rgb(0, 0, 100)',
                    size = 3,
                    opacity = 0.8
                )))

        # キック軌道用
        LABEL = [i for i in range(1, kick_frames)]
        data.append(go.Scatter3d(
            x =  [0],
            y =  [0],
            z =  [0],
            text = LABEL,
            mode = 'text+markers',
            marker = dict(
                color = ('rgb(155, 0, 0)'),
                size = 2,
                opacity = 0.8
        )))
        
        fig = go.FigureWidget(data)
                    
        # レイアウト
        fig.update_layout(
            height = 500,
            width = 500,
            margin = dict(
                l = 0,
                r = 0,
                b = 0,
                t = 0
            ),
            #カメラ位置
            scene_camera = dict(
                eye = dict(x = 2.0, y = 0, z = -1.0),
                up = dict(x = 1., y = 0., z = 0.)
            ),
            
            # 軸設定
            scene = dict(
                aspectmode = 'cube',
                #xaxis = dict(range = [0.0, 10.0]),
                #yaxis = dict(range = [-1.0, 1.0]),
                #zaxis = dict(range = [1.0, -1.0])),
                xaxis = dict(range = [-4.0, 4.0]),
                yaxis = dict(range = [-2.0, 2.0]),
                zaxis = dict(range = [-2.0, 2.0])),
                showlegend = False,
            )
        display(fig)
            
        # test.pyより
        with mp_pose.Pose(
            min_detection_confidence = 0.7,
            min_tracking_confidence = 0.3) as pose:

            # FocalLength(X,Y,Z)
            fx = w / (2 * math.tan(math.radians(69.4 / 2.)))
            fy = h / (2 * math.tan(math.radians(42.5 / 2.)))
            print("(fx, fy) = (%f, %f)" % (fx,fy))

            record_mode = False

            while True:     
                # フレーム待ち(Color & Depth)
                frames = pipeline.wait_for_frames()
                frames.keep()
                aligned_frames = align.process(frames)
                aligned_frames.keep()
                
                color_frame = aligned_frames.get_color_frame()
                depth_frame = aligned_frames.get_depth_frame()
                depth_image = np.asanyarray(depth_frame.get_data())

                if not depth_frame or not color_frame:
                    continue

                # imageをnumpy arrayに
                color_image = np.asanyarray(color_frame.get_data())
                depth_image = np.asanyarray(depth_frame.get_data())
                
                # To improve performance, optionally mark the image as not writeable to
                # pass by reference.
                if record_mode == False:
                    mp_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
                    mp_image.flags.writeable = False
                    results = pose.process(mp_image)

                    # Draw the pose annotation on the image.
                    image = color_image.copy()
                
                    # MediaPipeのランドマークを描画する
                    mp_drawing.draw_landmarks(
                        image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
                    # plotly表示用にランドマークを控えておく
                    try:
                        landmark = results.pose_landmarks.landmark
                    except:
                        landmark = None
                        
                # 取得されていればランドマークの座標を出力
                # 例として鼻（nose）の値を出力、fはフォーマット文字列で、書式を整えて出力できる
                # フォーマット文字列に変数の値を埋め込む場合は、{ }で囲んで埋め込む
                # distは辞書型、x, yはmediapipeから取得、それをもとにZ（深度）はRealSence2から取得
                # dist = {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].x,
                #      "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].y}
                # print(dist["x"],dist["y"])

                # CSVファイルの2行目以降の準備
                XYZlist = [count]

                # マーカーリストを初期化しておく
                marker_x = []
                marker_y = []
                marker_z = []
                #print("(X,Y,Z)=(%f,%f,%f)"%(X,Y,Z))

                # depth imageをカラーマップに変換
                depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(
                    depth_image, alpha = 0.08), cv2.COLORMAP_JET)

                # 各リストに必要なデータを控えておく（メモリ上に留めておく）
                if record_mode == True:
                    try:
                        # Plotly関係の演算をあとで行うため、ランドマークデータとdepth_frameも全てここで控えておく必要がある
                        depth_images.append(depth_image)
                        
                        # fps検証用列を追加して、CSV書き込み用XYZListを作成する
                        start = datetime.datetime.now()
                        XYZlist.extend([start])
                        XYZlists.append(copy.deepcopy(XYZlist))

                        # mediapipe画像とPlotly画像を追加する
                        color_images.append(copy.deepcopy(color_image))
                        depth_colormaps.append(copy.deepcopy(depth_colormap))
                        
                        # plotlyの保存に大きな負担がかかって30fps→15fpsになる
                        # ここではfigBatchUpdateもしないので、あとでやればよい
                        # plotly_figs.append(copy.deepcopy(fig))

                    except:
                        pass

                else:
                    # 画像表示
                    color_image_s = cv2.resize(color_image, (640, 360))
                    depth_colormap_s = cv2.resize(depth_colormap, (640, 360))
                    mp_image_s = cv2.resize(image, (640, 360))

                    images_h1 = np.hstack((color_image_s, depth_colormap_s))
                    images_h2 = np.hstack((mp_image_s, mp_image_s))
                    images_v = np.vstack((images_h1, images_h2))
                    cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
                    cv2.imshow('RealSense', images_v)
                
                    # record_modeがFalseのときは撮影ポジションを決めるために、
                    # ここでJupyter上にグラフを再描画する
                    try:
                        calcXYZ(XYZlist, depth_image, landmark) 
                        figBatchUpdate(fig)
                        
                    except:
                        #traceback.print_exc()
                        pass

                # ESCで終了
                key =  cv2.waitKey(1)
                if key & 0xff == 27: #ESC
                    record_mode = False
                    cv2.destroyAllWindows()
                    start = datetime.datetime.now()

                    print("ESC pressed. Recording stopped.")
                    print(start)

                    # メモリ上に留めておいた画像とテキストをディスクに保存する
                    print("Images saving started.")
                    frameCount = len(XYZlists)
                    print("%d frames" % frameCount)
                    
                    # plotlyのfigを取得する（figのままでもよいが一応）
                    pf = copy.deepcopy(fig)
                    
                    for i, XYZlist in enumerate(XYZlists):
                        # マーカーリストを初期化しておく
                        marker_x = []
                        marker_y = []
                        marker_z = []
                        
                        # To improve performance, optionally mark the image as not writeable to
                        # pass by reference.
                        mp_image = cv2.cvtColor(color_images[i], cv2.COLOR_BGR2RGB)
                        mp_image.flags.writeable = False
                        results = pose.process(mp_image)
                        
                        # Draw the pose annotation on the image.
                        mp_image.flags.writeable = True
                        mp_image = cv2.cvtColor(mp_image, cv2.COLOR_RGB2BGR)
                
                        # MediaPipeのランドマークを描画する
                        mp_drawing.draw_landmarks(
                            mp_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
                        # ランドマークデータからXYZを計算する
                        try:
                            landmark = results.pose_landmarks.landmark
                        except:
                            landmark = None

                        calcXYZ(XYZlist, depth_images[i], landmark)    

                        # fig.batch_update()でfigに反映する
                        figBatchUpdate(pf)

                        # CSVデータを保存する
                        writer.writerow(XYZlists[i])
                        
                        # 画像データを保存する
                        colorfilepath = dirname + "/color_%d.png" % i
                        cv2.imwrite(colorfilepath, color_images[i])

                        depthfilepath = dirname + "/depth_%d.png" % i
                        cv2.imwrite(depthfilepath, depth_colormaps[i])
                        
                        mpfilepath = dirname + "/mp_%d.png" % i
                        cv2.imwrite(mpfilepath, mp_image)
                        
                        plotlyfilepath = dirname2 + "/plotly_%d.png" % i
                        pf.write_image(plotlyfilepath)  
                        #b = pf.to_image(format="png")

                        print("%d " % (i + 1), end='')

                    print("")
                    print(datetime.datetime.now() - start)
                    print("Images saving complete.")
                    break

                elif key & 0xff == 13: # Enter 
                    print("Enter pressed. Recording started.")
                    record_mode = True
                    XYZlists = []
                    count = 0
                
                if record_mode == True:
                    count += 1
                
finally:
    # ストリーミング停止
    pipeline.stop()
    print("All complete.")  


FigureWidget({
    'data': [{'marker': {'color': 'rgb(0, 0, 100)', 'opacity': 0.8, 'size': 3},
              '…

(fx, fy) = (924.277380, 925.738464)
ESC pressed. Recording stopped.
2022-01-28 21:44:54.563236
Images saving started.
0 frames

0:00:00.044999
Images saving complete.
All complete.
